<a href="https://colab.research.google.com/github/cahya-wirawan/ConvAI.id/blob/main/ConvAI_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Domain Chatbot with Indonesian GPT-2

In [1]:
!nvidia-smi

Sat Feb 19 22:16:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation of Huggingface's Transformers and Facebook's ParlAI
We need to install Transformers and ParlAI to be able to finetune GPT-2 with dialog datasets. We have to install the latest version of ParlAI from its repository, since the support for arbitrary GPT-2 models has been just added few days ago (18. Feb. 2022) https://github.com/facebookresearch/ParlAI/pull/4360


In [2]:
%%capture

!pip install transformers
!python -m pip install git+https://github.com/facebookresearch/ParlAI.git

In [3]:
import parlai
parlai.__version__

'1.5.1'

## Download the Indonesian ConvAI2 and Empathetic Dialog Dataset
Currently we have two indonesian dialog dataset, translated from the english ConvAI2 (Persona Chat) and Empathetic Dialogues dataset. For this demo, we will only fine tune the empathetic dialog dataset. We download and store it in the directory "/content/data"

In [4]:
!mkdir -p data
# !cd data && wget https://cloud.uncool.ai/index.php/s/Sr66KMPeAs85Si4/download/ConvAI.tgz
# !cd data && tar xvzf ConvAI.tgz
!cd data && wget https://cloud.uncool.ai/index.php/s/kCg6QZoZqNgxoww/download/empatheticdialogues.tgz
!cd data && tar xvzf empatheticdialogues.tgz


--2022-02-19 22:18:52--  https://cloud.uncool.ai/index.php/s/kCg6QZoZqNgxoww/download/empatheticdialogues.tgz
Resolving cloud.uncool.ai (cloud.uncool.ai)... 144.76.103.67
Connecting to cloud.uncool.ai (cloud.uncool.ai)|144.76.103.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27380436 (26M) [application/x-compressed]
Saving to: ‘empatheticdialogues.tgz’

empatheticdialogues 100%[===================>]  26.11M  6.55MB/s    in 4.0s    

2022-02-19 22:18:57 (6.55 MB/s) - ‘empatheticdialogues.tgz’ saved [27380436/27380436]

empatheticdialogues/
empatheticdialogues/empatheticdialogues/
empatheticdialogues/empatheticdialogues/train.csv
empatheticdialogues/empatheticdialogues/valid.csv
empatheticdialogues/empatheticdialogues/test.csv
empatheticdialogues/.built


In [16]:
!head data/empatheticdialogues/empatheticdialogues/train.csv

conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
hit:0_conv:1,1,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,1,Saya ingat akan melihat kembang api bersama sahabat saya. Ini adalah pertama kalinya kami menghabiskan waktu sendirian bersama. Meskipun ada banyak orang_comma_ kami merasa seperti satu-satunya orang di dunia.,5|5|5_2|2|5,
hit:0_conv:1,2,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,0,Apakah ini teman yang Anda cintai_comma_ atau hanya sahabat?,5|5|5_2|2|5,
hit:0_conv:1,3,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,1,Ini adalah sahabat. Aku merindukannya.,5|5|5_2|2|5,
hit:0_conv:1,4,sentimental,Saya ingat pergi ke kembang api dengan sahabat saya. Ada banyak orang_comma_ tapi rasanya seperti kita di dunia.,0,Kemana dia

## Display Datasets Content
We can check the content of the dataset using the command line
`parlai display_data -t empathetic_dialogues --datapath data`
or inside the jupyter or a script. This is also valid for other commands such as training and evaluation.

In [10]:
from parlai.scripts.display_data import DisplayData
DisplayData.main(
  task='empathetic_dialogues',
  datapath='data',
  num_examples=10
)
# This is exactly the same as the following command line:
# parlai display_data -t empathetic_dialogues --datapath data --num-examples 10

00:06:04 | Opt:
00:06:04 |     allow_missing_init_opts: False
00:06:04 |     batchsize: 1
00:06:04 |     datapath: data
00:06:04 |     datatype: train:ordered
00:06:04 |     dict_class: None
00:06:04 |     display_add_fields: 
00:06:04 |     download_path: None
00:06:04 |     dynamic_batching: None
00:06:04 |     hide_labels: False
00:06:04 |     ignore_agent_reply: True
00:06:04 |     image_cropsize: 224
00:06:04 |     image_mode: raw
00:06:04 |     image_size: 256
00:06:04 |     init_model: None
00:06:04 |     init_opt: None
00:06:04 |     is_debug: False
00:06:04 |     loglevel: info
00:06:04 |     max_display_len: 1000
00:06:04 |     model: None
00:06:04 |     model_file: None
00:06:04 |     multitask_weights: [1]
00:06:04 |     mutators: None
00:06:04 |     num_examples: 2
00:06:04 |     override: "{'task': 'empathetic_dialogues', 'datapath': 'data', 'num_examples': 2}"
00:06:04 |     parlai_home: /usr/local/lib/python3.7/dist-packages
00:06:04 |     remove_political_convos: False

In [ ]:
# !parlai display_data -t convai2 --datapath data
# !parlai display_data -t empathetic_dialogues --datapath data


In [ ]:
from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task='empathetic_dialogues',
    datapath='data',
    model='hugging_face/gpt2',
    # 'indonesian-nlp/gpt2' is the indonesian GPT-2 small model
    model_name='indonesian-nlp/gpt2',
    model_file='models/empathetic_dialogues_small',
    # model_name='indonesian-nlp/gpt2-medium-indonesian',
    # model_file='models/empathetic_dialogues_medium',
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-3,
    optimizer='adam',
    # optimizer='adafactor'
    # warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    # max_train_time=120,
    num_epochs=1,
    validation_every_n_epochs=0.25,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=2, fp16=True,
    #fp16_impl='mem_efficient',
    
    # speeds up validation
    # skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)

### Download GPT-2 Medium Model finetuned on ConvAI2 and Empathetic Dialog datasets

In [2]:
# 
# !mkdir -p models
# !cd models && wget https://cdn-lfs.huggingface.co/cahya/persona_empathetic/05de188749c3ab1ae93978044116c4acc3b31efb9bc3b769a18f11219830db18 -O persona_empathetic.tar
# !cd models && tar xvf persona_empathetic.tar

--2022-02-19 23:08:06--  https://cdn-lfs.huggingface.co/cahya/persona_empathetic/05de188749c3ab1ae93978044116c4acc3b31efb9bc3b769a18f11219830db18
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.224.154.93, 13.224.154.121, 13.224.154.37, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.224.154.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5703321600 (5.3G) [binary/octet-stream]
Saving to: ‘persona_empathetic.tar’

persona_empathetic. 100%[===================>]   5.31G  13.9MB/s    in 4m 11s  

2022-02-19 23:12:19 (21.6 MB/s) - ‘persona_empathetic.tar’ saved [5703321600/5703321600]

persona_empathetic/
persona_empathetic/medium.trainstats
persona_empathetic/medium
persona_empathetic/medium.dict
persona_empathetic/medium.dict.opt
persona_empathetic/medium.opt


In [21]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='empathetic_dialogues',
    datapath='data',
    model_file='models/empathetic_dialogues_small',
    # model_file='models/persona_empathetic/medium',
    num_examples=15,
    skip_generation=False,
    inference='beam',
    beam_size=10,
    beam_context_block_ngram=3,
    beam_block_ngram=3,
    beam_min_length=15
)

00:27:23 | Overriding opt["inference"] to beam (previously: greedy)
00:27:23 | Overriding opt["beam_size"] to 10 (previously: 1)
00:27:23 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
00:27:23 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
00:27:23 | Overriding opt["beam_min_length"] to 15 (previously: 1)
00:27:23 | Using CUDA


Some weights of the model checkpoint at indonesian-nlp/gpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


00:27:33 | Total parameters: 124,441,344 (124,441,344 trainable)
00:27:33 | Loading existing model params from models/empathetic_dialogues_small
00:27:35 | creating task(s): empathetic_dialogues
[EmpatheticDialoguesTeacher] Only use experiencer side? True, datatype: valid
00:27:35 | Opt:
00:27:35 |     adafactor_eps: '[1e-30, 0.001]'
00:27:35 |     adam_eps: 1e-08
00:27:35 |     add_p1_after_newln: False
00:27:35 |     add_special_tokens: True
00:27:35 |     add_start_token: False
00:27:35 |     aggregate_micro: False
00:27:35 |     allow_missing_init_opts: False
00:27:35 |     batchsize: 2
00:27:35 |     beam_block_full_context: True
00:27:35 |     beam_block_list_filename: None
00:27:35 |     beam_block_ngram: 3
00:27:35 |     beam_context_block_ngram: 3
00:27:35 |     beam_delay: 30
00:27:35 |     beam_length_penalty: 0.65
00:27:35 |     beam_min_length: 15
00:27:35 |     beam_size: 10
00:27:35 |     betas: '[0.9, 0.999]'
00:27:35 |     bpe_add_prefix_space: None
00:27:35 |     bpe_

/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1749: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


- - - NEW EPISODE: empathetic_dialogues- - -
Hari ini, saat saya berangkat kerja di pagi hari, ban saya pecah di tengah jalan yang sibuk. Itu membuatku takut!
    labels: Apakah kamu baik-baik saja sekarang?
     model: Oh tidak! Saya harap semuanya berjalan dengan baik untuk Anda. Saya harap Anda baik-baik saja!
Ya, saya baik-baik saja sekarang, tetapi dengan luka ringan.
    labels: Keren :) Apakah mobil Anda banyak rusak?
     model: Saya harap semuanya berjalan dengan baik untuk Anda. Saya harap Anda merasa lebih baik.
- - - NEW EPISODE: empathetic_dialogues- - -
Beberapa minggu yang lalu, saya sedang berjalan melalui lorong saya, mengurus urusan saya sendiri, ketika tiba-tiba sebuah tangan mengulurkan tangan dari bawah meja dan meraih pergelangan kaki saya. Saya sangat terharu. Saya pikir saya didapat. Ternyata, itu anak saya.
    labels: Itu lucu, semoga dia tidak memberimu serangan jantung.
     model: Oh tidak! Itu mengerikan. Saya harap semuanya berjalan dengan baik pada akhir

In [ ]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # model_file='models/empathetic_dialogues_small',
    model_file='models/persona_empathetic/medium',
    inference='beam',
    beam_size=10,
    beam_context_block_ngram=3,
    beam_block_ngram=3,
    beam_min_length=15
)

In [2]:
!parlai interactive --model "hugging_face/gpt2" -mf "models/persona_empathetic/medium" --inference beam --beam-size 10 --beam-context-block-ngram 3 --beam-block-ngram 3 --beam-min-length 25 

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
00:39:17 | WARNING: this model is in beta and the API is subject to change.
00:39:17 | Overriding opt["inference"] to beam (previously: greedy)
00:39:17 | Overriding opt["beam_size"] to 10 (previously: 1)
00:39:17 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
00:39:17 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
00:39:17 | Overriding opt["beam_min_length"] to 25 (previously: 1)
00:39:17 | Using CUDA
00:39:22 | Gpt2: full interactive mode on.
Some weights of the model checkpoint at flax-community/gpt2-medium-indonesian were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another 

In [2]:
!cp -r /content/models /content/drive/MyDrive/ConvAI/

In [9]:
!cp -r /content/drive/MyDrive/ConvAI/models /content

In [3]:
ls -lh /content/drive/MyDrive/ConvAI/models

total 9.6G
-rw------- 1 root root 250M Feb 19 17:17 empathetic_dialogues_01
-rw------- 1 root root 506K Feb 19 17:17 empathetic_dialogues_01.dict
-rw------- 1 root root 4.4K Feb 19 17:17 empathetic_dialogues_01.dict.opt
-rw------- 1 root root 3.8K Feb 19 17:17 empathetic_dialogues_01.opt
-rw------- 1 root root  353 Feb 19 17:17 empathetic_dialogues_01.test
-rw------- 1 root root  23K Feb 19 17:17 empathetic_dialogues_01.trainstats
-rw------- 1 root root  354 Feb 19 17:17 empathetic_dialogues_01.valid
-rw------- 1 root root 4.7G Feb 19 17:15 empathetic_dialogues_medium
-rw------- 1 root root 506K Feb 19 17:14 empathetic_dialogues_medium.dict
-rw------- 1 root root 4.5K Feb 19 17:15 empathetic_dialogues_medium.dict.opt
-rw------- 1 root root 4.7G Feb 19 17:16 empathetic_dialogues_medium_en
-rw------- 1 root root 506K Feb 19 17:17 empathetic_dialogues_medium_en.dict
-rw------- 1 root root 4.5K Feb 19 17:17 empathetic_dialogues_medium_en.dict.opt
-rw------- 1 root root 3.9K Feb 19 17:17 em